In [ ]:
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [ ]:
data = pd.read_csv('/Users/veda/Downloads/Bank_Personal_Loan_Modelling.csv')

In [ ]:
data.head()

In [ ]:
data.drop(["ID", "ZIP Code"],axis=1,inplace=True)

In [ ]:
x = data['Personal Loan']
data = data.drop(['Personal Loan'],axis=1)
data = data.join(x)

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
enc = OneHotEncoder()
y = enc.fit_transform(y[:, np.newaxis]).toarray()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=69)
(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
def create_custom_model(nodes, n=1, name='model'):
    model = Sequential(name=name)
    for i in range(n):
        model.add(Dense(nodes, input_dim=13, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [ ]:
n_layers = 1
model = create_custom_model(4, n_layers)
model.summary()

score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def fitness_function(weights):
    y_pred = model(X_train)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    accuracy = (y_pred == Y_train).sum().float().item() / len(test.dataset)
    return accuracy

In [ ]:
def cultural_algorithm(num_weights, pop_size, num_iter):
    population = []
    for i in range(pop_size):
        weights = model.get_weights()
        for j in range(len(weights)):
            weights[j] += np.random.normal(0, 0.1, size=weights[j].shape)
        fitness = fitness_function(weights)
        population.append({'weights': weights, 'fitness': fitness})

    population = sorted(population, key=lambda x: x['fitness'], reverse=True)

    cultural_pool = population[:int(pop_size/2)]

    for i in range(num_iter):
        new_individual = {}
        weights = model.get_weights()
        for j in range(len(weights)):
            if random.random() < 0.5:
                new_individual['weights'][j] = cultural_pool[0]['weights'][j]
            else:
                new_individual['weights'][j] = cultural_pool[1]['weights'][j]
            new_individual['weights'][j] += np.random.normal(0, 0.1, size=new_individual['weights'][j].shape)
        new_individual['fitness'] = fitness_function(new_individual['weights'])

        replace_index = random.randint(0, pop_size-1)
        population[replace_index] = new_individual

        if new_individual['fitness'] > cultural_pool[-1]['fitness']:
            cultural_pool[-1] = new_individual

        population = sorted(population, key=lambda x: x['fitness'], reverse=True)

    return population[0]['weights']


In [ ]:
best_weights = cultural_algorithm(num_weights=10, pop_size=50, num_iter=100)
print("Best weights found:", best_weights)

In [ ]:
model.set_weights(best_weights)

loss, accuracy = model.evaluate(X_val, y_val)

print("Accuracy:", accuracy)